In [1]:
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Activation
from keras.activations import softmax as Softmax
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from nltk.translate.bleu_score import corpus_bleu
from keras import backend as K
import numpy as np
import unicodedata
import re

Using TensorFlow backend.


In [0]:
BATCH_SIZE = 128  # Batch size for training
NUM_SAMPLES = 12500
EPOCHS = 50
OPTIMIZER = "adam"
EMBED_DIM = 100
HIDDEN_DIM = 256
DATA_PATH = 'deu.txt'

In [0]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

In [0]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile, 'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.", len(model), " words loaded!")
    return model

In [0]:
def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())

    # creating a space between words and punctation following it
    # eg: "he is a boy." => "he is a boy ."
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()

    # adding a start and an end token to the sequence
    # so that the model know when to start and stop predicting
    w = "\t " + w + " \n"
    return w

In [6]:
eng_embedding = loadGloveModel('glove.6B.100d.txt')

Loading Glove Model
Done. 400000  words loaded!


In [0]:
input_texts = []
target_texts = []
target_chars = set()

with open(DATA_PATH, 'r', encoding='utf-8') as f:
    lines = f.read().split("\n")
    for line in lines[:NUM_SAMPLES]:
        input_text, target_text = line.split('\t')
        input_text = preprocess_sentence(input_text)
        target_text = preprocess_sentence(target_text)
        input_texts.append(input_text)
        target_texts.append(target_text)
        target_chars.update(list(target_text))

target_chars = sorted(list(target_chars))

In [0]:
# get attributes from data
max_input_seqlen = max([len(txt.split()) for txt in input_texts])
max_target_seqlen = max([len(txt) for txt in target_texts])
target_token_size = len(target_chars)

In [0]:
# get decoder data
targchars2idx = dict([(char, i) for i, char in enumerate(target_chars)])
idx2targchars = dict((i, char) for char, i in targchars2idx.items())
decoder_data = np.zeros(
    shape=(NUM_SAMPLES, max_target_seqlen, target_token_size))
decoder_target_data = np.zeros(
    shape=(NUM_SAMPLES, max_target_seqlen, target_token_size))

for textIdx, text in enumerate(target_texts):
    for idx, char in enumerate(text):
        c2idx = targchars2idx[char]
        decoder_data[textIdx, idx, c2idx] = 1
        if idx > 0:
            decoder_target_data[textIdx, idx - 1, c2idx] = 1

In [0]:
# get encoder data
encoder_data = []
for text in input_texts:
    tmp = []
    for word in text.split():
        embed = np.random.randn(EMBED_DIM)
        if word in eng_embedding:
            embed = eng_embedding[word]
        tmp.append(embed)
    encoder_data.append(tmp)
encoder_data = pad_sequences(encoder_data, max_input_seqlen, padding="post")

In [11]:
decoder_data.shape

(12500, 77, 33)

In [12]:
decoder_target_data.shape

(12500, 77, 33)

In [13]:
encoder_data.shape

(12500, 7, 100)

In [0]:
# construct model
encoder_inputs = Input(shape=(max_input_seqlen, EMBED_DIM))
encoder_lstm = LSTM(HIDDEN_DIM, return_state=True, name="encoder_lstm")
_, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, target_token_size))
decoder_lstm = LSTM(HIDDEN_DIM, return_sequences=True,
                    return_state=True, name="decoder_lstm")
decoder_outputs, _, _ = decoder_lstm(
    decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(
    target_token_size, activation="softmax", name="decoder_dense")
decoder_outputs = decoder_dense(decoder_outputs)

In [15]:
# define training model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer=OPTIMIZER,
              loss='categorical_crossentropy', metrics=["acc"])
print(model.summary())
filename = 'seq2seq_keras.h5'
checkpoint = ModelCheckpoint(
    filename, verbose=1, save_best_only=True, mode='min')
# checkpoint = ModelCheckpoint(filename, verbose=1, mode='min')
model.fit([encoder_data, decoder_data], decoder_target_data,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS, callbacks=[checkpoint], validation_split=0.2)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 7, 100)       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 33)     0                                            
__________________________________________________________________________________________________
encoder_lstm (LSTM)             [(None, 256), (None, 365568      input_1[0][0]                    
__________________________________________________________________________________________________
decoder_lstm (LSTM)             [(None, None, 256),  296960      input_2[0][0]                    
                                                                 encoder_lstm[0][1]               
          

/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer decoder_lstm was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'encoder_lstm/while/Exit_3:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'encoder_lstm/while/Exit_4:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/50
10000/10000 [==============================] - 67s 7ms/step - loss: 0.5861 - acc: 0.1337 - val_loss: 0.5700 - val_acc: 0.1659

Epoch 00002: val_loss improved from 0.78080 to 0.56997, saving model to seq2seq_keras.h5
Epoch 3/50
10000/10000 [==============================] - 67s 7ms/step - loss: 0.4710 - acc: 0.1612 - val_loss: 0.5045 - val_acc: 0.1822

Epoch 00003: val_loss improved from 0.56997 to 0.50455, saving model to seq2seq_keras.h5
Epoch 4/50
10000/10000 [==============================] - 67s 7ms/step - loss: 0.4226 - acc: 0.1719 - val_loss: 0.4682 - val_acc: 0.1910

Epoch 00004: val_loss improved from 0.50455 to 0.46819, saving model to seq2seq_keras.h5
Epoch 5/50
10000/10000 [==============================] - 66s 7ms/step - loss: 0.3933 - acc: 0.1801 - val_loss: 0.4399 - val_acc: 0.1984

Epoch 00005: val_loss improved from 0.46819 to 0.43993, saving model to seq2seq_keras.h5
Epoch 6/50
10000/10000 [==============================] - 61s 6ms/step - loss: 0.3712 - acc:

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(HIDDEN_DIM,))
decoder_state_input_c = Input(shape=(HIDDEN_DIM,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
# decoder_outputs = (BATCH_SIZE, seqlen, HIDDEN_DIM)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
# decoder_outputs = (BATCH_SIZE, seqlen, target_token_size)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [0]:
def decode(input_seq):
    states = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, target_token_size))
    target_seq[0, 0, targchars2idx['\t']] = 1
    stop_condition = False
    prediction = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states)
        sampled_token_idx = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2targchars[sampled_token_idx]
        prediction += sampled_char

        if (sampled_char == '\n' or len(prediction) > max_target_seqlen):
            stop_condition = True

        target_seq = np.zeros((1, 1, target_token_size))
        target_seq[0, 0, sampled_token_idx] = 1
        states = [h, c]

    return prediction

In [18]:
actual, predicted = list(), list()

for index in range(1000):
    input_seq = encoder_data[index]
    input_seq = np.expand_dims(input_seq, axis=0)
    actual.append(target_texts[index].split())
    prediction = decode(input_seq)
    predicted.append(prediction.split())
    print('-')
    print("Input sentence: ", input_texts[index])
    print("Translation: ", prediction)
    print("Truth value: ", target_texts[index])

print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
print('BLEU-2: %f' %
      corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
print('BLEU-3: %f' %
      corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
print('BLEU-4: %f' %
      corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

-
Input sentence:  	 hi . 

Translation:   hallo ! 

Truth value:  	 hallo ! 

-
Input sentence:  	 hi . 

Translation:   hallo ! 

Truth value:  	 gru gott ! 

-
Input sentence:  	 run ! 

Translation:   lauf ! 

Truth value:  	 lauf ! 

-
Input sentence:  	 fire ! 

Translation:   feuer ! 

Truth value:  	 feuer ! 

-
Input sentence:  	 help ! 

Translation:   unterschinden ! 

Truth value:  	 hilfe ! 

-
Input sentence:  	 help ! 

Translation:   unterschinden ! 

Truth value:  	 zu hulf ! 

-
Input sentence:  	 stop ! 

Translation:   stopp ! 

Truth value:  	 stopp ! 

-
Input sentence:  	 wait ! 

Translation:   warte ! 

Truth value:  	 warte ! 

-
Input sentence:  	 go on . 

Translation:   mach dich fort ! 

Truth value:  	 mach weiter . 

-
Input sentence:  	 hello ! 

Translation:   hallo ! 

Truth value:  	 hallo ! 

-
Input sentence:  	 i ran . 

Translation:   ich war nach . 

Truth value:  	 ich rannte . 

-
Input sentence:  	 i see . 

Translation:   ich will . 

Truth 

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEU-1: 0.241094
BLEU-2: 0.491014
BLEU-3: 0.652614
BLEU-4: 0.700724


In [0]:
def predict_sent(input_text):
    input_seq = []
    tmp = []
    text = preprocess_sentence(input_text)
    for word in text.split():
        embed = np.random.randn(EMBED_DIM)
        if word in eng_embedding:
            embed = eng_embedding[word]
        tmp.append(embed)
    input_seq.append(tmp)
    input_seq = pad_sequences(input_seq, max_input_seqlen, padding="post")
    states = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, target_token_size))
    target_seq[0, 0, targchars2idx['\t']] = 1
    target_text = ''
    stop_condition = False
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states)

        sampled_token_idx = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2targchars[sampled_token_idx]
        target_text += sampled_char

        if sampled_char == '\n' or len(target_text) >= max_target_seqlen:
            stop_condition = True

        target_seq = np.zeros((1, 1, target_token_size))
        target_seq[0, 0, sampled_token_idx] = 1

        states = [h, c]
    return target_text.strip()

In [22]:
predict_sent("She is beautiful")

'sie ist star .'